In [1]:
import pandas as pd
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.shared import RGBColor
from docx.shared import Cm
from docx.enum.table import WD_ROW_HEIGHT_RULE
import math
import re


In [2]:
def create_page(document):
    
    table = document.add_table(11,4)
    table.style = 'Table Grid'
    
    return table

In [3]:
def merge_cells(table, row, max_cell, start_cell=0):
    
    merge_anchor = table.cell(row, start_cell)
    for i in range(start_cell+1, max_cell+1):
        merge_anchor.merge(table.cell(row, i))
    
    return merge_anchor

In [4]:
def blue_cell(cell):
    
    # Set a cell background (shading) color to RGB D9D9D9.
    shading_elm = parse_xml(r'<w:shd {} w:fill="4682B4"/>'.format(nsdecls('w')))
    cell._tc.get_or_add_tcPr().append(shading_elm)

In [5]:
def write_white(cell, text):
    
    text = cell.paragraphs[0].add_run(text)
    white = RGBColor(255,255,255)
    text.font.color.rgb = white

In [6]:
def white_on_blue(cell, text):
    
    blue_cell(cell)
    write_white(cell, text)

In [7]:
def white_row(table, row_num):
    
    merge_cells(table, row_num, max_cell=3)
    row = table.rows[row_num]
    row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
    row.height = Cm(.3)


In [8]:
def add_eixo(table, data_row):
    
    key = merge_cells(table, 0, 1)
    white_on_blue(key, 'Eixo:')
    
    val = merge_cells(table, 0, 3, start_cell=2)
    val.text = data_row['Eixo proposto']


In [9]:
def add_programa(table, data_row):
    
    key = merge_cells(table, 1, 1)
    white_on_blue(key, 'Programa/Projeto:')
    
    val = merge_cells(table, 1, 3, start_cell=2)
    val.text = data_row['projeto']


In [10]:
def add_secretaria(table, data_row):
    
    key = merge_cells(table, 2, 1)
    white_on_blue(key, 'Secretaria:')
    
    val = merge_cells(table, 2, 3, start_cell=2)
    val.text = data_row['secretaria']

In [11]:
def add_detalhamento(table, data_row):
    
    val = merge_cells(table, 3, 3, start_cell=0)
    
    texto = data_row['estagio_atual'] + '\n' + data_row['objetivo'] 
    val.text = texto

In [12]:
def add_andamento_acao_futura(table, data_row):
    
    key = table.cell(4,0)
    white_on_blue(key, 'Em andamento:')
    
    val = table.cell(4,1)
    val.text = data_row['andamento']
    
    
    key = table.cell(4,2)
    white_on_blue(key, 'Ação futura:')
    
    val = table.cell(4,3)
    val.text = data_row['acao_futura']

In [13]:
def add_beneficiarios(table, data_row):
    
    key = merge_cells(table, 5, 3)
    white_on_blue(key, 'Beneficiários:')
    
    val = merge_cells(table, 6, 3)
    val.text = data_row['beneficiarios']

In [14]:
def add_resultados_esperados(table, data_row):
    
    key = merge_cells(table, 7, 3)
    white_on_blue(key, 'Resultados Esperados e Impactos:')
    
    val = merge_cells(table, 8, 3)
    val.text = data_row['resultados_e_impactos']

In [15]:
def add_vinculacao_pdm(table, data_row):
    
    key = merge_cells(table, 9, 1)
    white_on_blue(key, 'Vinculação PDM:')
    
    val = merge_cells(table, 9, 3, start_cell=2)
    val.text = ''

In [16]:
def add_vinculacao_ods(table, data_row):
    
    key = merge_cells(table, 10, 1)
    white_on_blue(key, 'Vinculação ODS:')
    
    val = merge_cells(table, 10, 3, start_cell=2)
    val.text = ''

In [17]:
def add_anotacoes(document, data_row):
    
    obs_sepep = f"Obs SEPEP: {data_row['SEPEP - OBS']}"
    obs_sepe = f"Obs SEPE: {data_row['SEPE - OBS']}"
    
    document.add_paragraph('')#para separar da tabela
    document.add_paragraph(obs_sepep)
    document.add_paragraph(obs_sepe)

In [18]:
def write_project(document, data_row):
    
    table = create_page(document)
    
    add_eixo(table, data_row)
    add_programa(table, data_row)
    add_secretaria(table, data_row)
    add_detalhamento(table, data_row)
    add_andamento_acao_futura(table, data_row)
    add_beneficiarios(table, data_row)
    add_resultados_esperados(table, data_row)
    add_anotacoes(document, data_row)
    add_vinculacao_pdm(table, data_row)
    add_vinculacao_ods(table, data_row)
    
    document.add_page_break()

In [19]:
df = pd.read_excel('original_data/novo_formato/Sistematização formulários.xlsx')

/home/h-pgy/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [20]:
def filtrar_ok_sepep(df):
    
    ok_sepep = df['SEPEP - ADERÊNCIA AO ESCOPO DO PLANO DE RETOMADA? (SIM/NÃO)'].str.lower()=='sim'
    
    return df[ok_sepep].copy()

In [21]:
def dropar_col_zuadas(df):
    
    dropar = [col for col in df.columns 
             if col.startswith('Unnamed')]
    
    df = df.drop(dropar, axis=1).copy()
    
    return df

In [22]:
def ordenar(df):
    
    col_ordem = 'Numeração 10.08'

    df.sort_values(by=col_ordem, inplace=True)

In [23]:
def cols_to_str(df):
    
    df.fillna('', inplace=True)
    for col in df.columns:
        df[col] = df[col].apply(str)

In [24]:
def limpar_df_geral(df):
    
    df = filtrar_ok_sepep(df)
    df = dropar_col_zuadas(df)
    
    ordenar(df)
    cols_to_str(df)
    
    return df

In [25]:
def separar_dfs(df):
    
    eixos = df['Eixo proposto'].unique()
    dfs =[]
    for eixo in eixos:
        filtro = df['Eixo proposto']==eixo
        filtrado = df[filtro].copy().reset_index(drop=True)
        dfs.append(filtrado)
        
    return dfs

In [26]:
df = limpar_df_geral(df)

In [27]:
dfs = separar_dfs(df)

In [28]:
for i, df in enumerate(dfs):
    document = Document()
    for row_num, row in df.iterrows():
        print(row_num)
        write_project(document, row)
    document.save(f'teste_novo_formato{i}.docx')

0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
0
1
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
